In [2]:
import requests
import pandas as pd
import numpy as np
import folium
from bs4 import BeautifulSoup

In [3]:
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
page.status_code

200

In [4]:
import lxml
soup = BeautifulSoup(page.content, "lxml")
table = soup.find('table', {"class":"wikitable sortable"})
trs = table.find_all('tr')

postcode = []
borough = []
neighbourhood = []

for tr in trs:
    tds = tr.find_all('td')
    if tds:
        postcode.append(tds[0].text.strip())
        borough.append(tds[1].text.strip())
        neighbourhood.append(tds[2].text.strip())
        
df_html = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_html .columns=['PostalCode', 'Borough', 'Neighborhood']
df_html.loc[df_html['Borough'] == "Not assigned", 'Borough'] = np.nan
df_html.loc[df_html['Neighborhood'] == "Not assigned", 'Neighborhood'] = np.nan
df_html.head()

,PostalCode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
df_html.dropna(subset=['Borough'], inplace=True)

In [6]:
not_empty_neighborhood = df_html[df_html['Neighborhood'].isna()].shape[0]
print('Count of Neighborhood column is empty: {}'.format(not_empty_neighborhood))

Count of Neighborhood column is empty: 1


In [7]:
df_html[df_html['Neighborhood'].isna()]

,PostalCode,Borough,Neighborhood
9,M9A,Queen's Park,NaN


In [8]:
df_html['Neighborhood'].fillna(df_html['Borough'], inplace=True)
not_empty_neighborhood = df_html[df_html['Neighborhood'].isna()].shape[0]
print('Count of Neighborhood column is empty: {}'.format(not_empty_neighborhood))

Count of Neighborhood column is empty: 0


In [9]:
df_html[df_html['Borough']=="Queen's Park"]

,PostalCode,Borough,Neighborhood
9,M9A,Queen's Park,Queen's Park


In [10]:
df_postcodes =pd.DataFrame( df_html.groupby(['PostalCode','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)]))
df_postcodes.reset_index(inplace=True)
df_postcodes.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df_html.dropna(subset=['Borough'], inplace=True)

In [12]:
not_empty_neighborhood = df_html[df_html['Neighborhood'].isna()].shape[0]
print('Count of Neighborhood column is empty: {}'.format(not_empty_neighborhood))

Count of Neighborhood column is empty: 0


In [13]:
print('The shape is:',df_postcodes.shape)

The shape is: (103, 3)


In [14]:
df_postcodes.to_csv('Toronto_Postcodes.csv')

In [15]:
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df_neighborhoods = pd.read_csv('Toronto_Postcodes.csv',index_col=[0])
df_neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_neighborhoods.rename(columns={'PostalCode': 'PostalCode'}, inplace=True)

In [18]:
df_neighborhoods_coordinates = pd.merge(df_neighborhoods, df_coordinates, on='PostalCode')
df_neighborhoods_coordinates.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
df_neighborhoods_coordinates[(df_neighborhoods_coordinates['PostalCode']=='M5G') |
                             (df_neighborhoods_coordinates['PostalCode']=='M2H') ]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [20]:
df_neighborhoods_coordinates.to_csv('Toronto_Postcodes_2.csv')

In [21]:
df = pd.read_csv('Toronto_Postcodes_2.csv', index_col=0)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [22]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [23]:
df.rename(columns={'Neighborhood': 'Neighborhood'}, inplace=True)
df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           11
Mississauga          1
North York          24
Queen's Park         1
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [24]:
df_toronto = df[df['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [25]:
print(df_toronto.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


In [26]:
boroughs = df_toronto['Borough'].unique().tolist()
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717948, -79.38987324871795


In [27]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))
    
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'],
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

OK. There are data I use to report.